<a href="https://colab.research.google.com/github/yiw008/nondet-project/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

In [ ]:
import json

input_file = 'mydata.jsonl'

with open(input_file, 'r') as file:
  lines = file.readlines()

cleaned_lines = []
rec = ''
for line in lines:
  if line == '}\n' or line == '}':
    rec += line
    cleaned_lines.append(rec)
    rec = ''
  elif line.endswith(']}\n'):
    cleaned_lines.append(line)
  else:
    rec += line.replace('\n', '')

In [ ]:
true_lines = []
false_lines = []

for line in cleaned_lines:
  data = json.loads(line)
  for message in data['messages']:
    if message['role'] == 'assistant':
      if message['content'] == 'True':
        true_lines.append(json.dumps(data) + '\n')
      elif message['content'] == 'False':
        false_lines.append(json.dumps(data) + '\n')

In [ ]:
true_size = len(true_lines)
false_size = len(false_lines)
total_size = true_size + false_size
half_training_size = min(int(total_size * 0.8 / 2), true_size, false_size)
training_size = half_training_size * 2
test_size = total_size - training_size

In [ ]:
import random

true_training_set = random.sample(true_lines, half_training_size)
false_training_set = random.sample(false_lines, half_training_size)
true_test_set = [line for line in true_lines if line not in true_training_set]
false_test_set = [line for line in false_lines if line not in false_training_set]
training_set = true_training_set + false_training_set
test_set = true_test_set + false_test_set

print(f'True in Training set: {len(true_training_set)}')
print(f'False in Training set: {len(false_training_set)}')
print(f'True in Test set: {len(true_test_set)}')
print(f'False in Test set: {len(false_test_set)}')
print(f'Total in Training set: {len(training_set)}')
print(f'Total in Test set: {len(test_set)}')
print(f'Total True: {len(true_lines)}')
print(f'Total False: {len(false_lines)}')
print(f'Total: {total_size}')

True in Training set: 5
False in Training set: 5
True in Test set: 5
False in Test set: 0
Total in Training set: 10
Total in Test set: 5
Total True: 10
Total False: 5
Total: 15


In [ ]:
with open('training_set.jsonl', 'w') as file:
  file.writelines(training_set)

with open('test_set.jsonl', 'w') as file:
  file.writelines(test_set)